**Question: Are there differences in medications prescribed off-label in patients diagnosed with Asthma during childhood vs during adulthood?**
* Find age of diagosis for Asthma patients
* Find medications (and RxNorm codes for meds) taken by Asthma patients
* Find all diagnoses of Asthma patients
* Classify medications taken by each patient as on- or off- label drug use
* Find medication classes for medications taken by Asthma patients
* Compare medication classes taken by patients diagosed with Asthma during childhood vs during adulthood

In [1]:
## All the imports we need
from urllib2 import Request, urlopen, URLError
from urllib import quote_plus

import mysql.connector

import pprint, json, requests

from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from greentranslator.api import GreenTranslator

import dateutil

**Step 1: Find patients diagnosed with Asthma as a child (Group 1)**

In [ ]:
## Find HUSH+ patients diagnosed as a child

**Step 2: Find patients diagnosed with Asthma as an adult (Group 2)**

In [ ]:
## Find HUSH+ patients diagnosed as adults

** Step 3: Find medications for patients in Group 1 and Group 2 **

In [2]:
## Pull meds for HUSH+ patients
meds = {}
for x in HUSHpluspatients:
    medList = x['medList']
    # Collect the unique meds
    for m in medList.keys():
        found = False
        try:
            meds[medList[m]] = meds[medList[m]]+1
        except KeyError:
            meds[medList[m]] = 1
#pprint.pprint(meds)

NameError: name 'HUSHpluspatients' is not defined

** Step 4: Find RxNorm codes for meds prescribed in patients from Group 1 and Group 2 **

In [3]:
## For a given medication string get NCBO annotations
## We let NCBO match any ontology since just using RxNORM doesn't
## always give us just the drug name (e.g., "CLINDAMYCIN 15 MG/ML ORAL SOLUTION" is
## a valid RxNORM term)
def med2rxnorm(txt):
    url = 'http://data.bioontology.org/annotator?text=%s&apikey=b792dd1b-cdc2-4cc8-aaf2-4fa4fbf47e4e'
    txt = urlopen(url % quote_plus(txt)).read()
    resp = json.loads(txt)
    if len(resp) == 0: return([])
    annos = []
    for aresp in resp:
        annos.extend([ x['text'] for x in aresp['annotations'] ])
    ##annos = filter(lambda x: not any(d in x for d in'0123456789'), annos)
    return(annos)
print med2rxnorm("CLINDAMYCIN 15 MG/ML ORAL SOLUTION")        

[u'CLINDAMYCIN 15 MG/ML ORAL SOLUTION', u'CLINDAMYCIN 15 MG/ML', u'CLINDAMYCIN', u'ORAL SOLUTION']


In [4]:
## Get RxNORM codes for medication strings
medrxnorm = {}
for med in meds.keys():
    if med is None: continue    
    annos = med2rxnorm(med)
    print 'Processing %s and found %d annotations' % (med, len(annos))
    medrxnorm[med] = {'count':meds[med], 'annos':annos}    

** Step 5: Find all indications (as ICD-10 codes) for any of the drugs prescribed in patients from Group 1 and Group 2 **

In [6]:
## Get ICD-10 codes for any med seen for Group 1 or Group 2

** Step 6: Find all diagnoses of Asthma patients in Group 1 and Group 2**

In [ ]:
## Find all diagnoses of HUSH+ patients in Group 1
## Find all diagnoses of HUSH+ patients in Group 2

**Step 7: For both groups, classify medications taken by each patient as on- or off- label drug use (patient by drug matrix)**

In [ ]:
## Create patient by drug matrix

** Step 8: Create patient by drug class matrix **

In [ ]:
## Create patient by drug class matrix

**Step 9: Assess off-label drug use in childhood-onset vs adult-onset Asthma patients **

In [8]:
## Compare patient x drug class for Group 1 vs Group 2